<a href="https://colab.research.google.com/github/ireneabby2003/ML-For-Beginners/blob/main/streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import StandardScaler

# Load model
tabnet = TabNetClassifier()
tabnet.load_model("tabnet_model.zip")

# Load your test data
X_test = pd.read_csv("X_tab_test.csv")  # Save this from your notebook
y_test = pd.read_csv("y_tab_test.csv")

# Inference
preds_proba = tabnet.predict_proba(X_test)
preds_class = preds_proba.argmax(axis=1)

st.title("🧠 TabNet Model Interpretability")

index = st.number_input("Choose sample index", min_value=0, max_value=len(X_test)-1, step=1)

sample = X_test.iloc[[index]]
true_label = y_test.iloc[index]
pred_label = preds_class[index]
prob = preds_proba[index][pred_label]

st.write(f"### 🎯 Prediction: {pred_label} with probability {prob:.4f}")
st.write(f"### 🏷️ True label: {true_label}")

# Plot explanation
explain_matrix, masks = tabnet.explain(sample.values)

# Step-by-step attention heatmap
fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(masks[0], cmap="YlGnBu", xticklabels=X_test.columns, yticklabels=[f"Step {i}" for i in range(masks[0].shape[0])], ax=ax)
plt.title("Step-by-Step Attention")
st.pyplot(fig)

# Aggregated attention
agg_attn = masks[0].sum(axis=0)
sorted_idx = np.argsort(agg_attn)[::-1]
sorted_features = np.array(X_test.columns)[sorted_idx]
sorted_values = agg_attn[sorted_idx]

fig2, ax2 = plt.subplots()
sns.barplot(x=sorted_values, y=sorted_features, palette="viridis", ax=ax2)
plt.title("Aggregated Feature Importance")
st.pyplot(fig2)
